In [ ]:

import pandas as pd
import numpy as np

import tensorflow as tf

repo_path = ".."
import sys
sys.path.append(f"{repo_path}/code/")



In [ ]:
from benchmarking import benchmark_likelihood
from benchmarking import benchmark_optimal

optimal_path = "optimal.csv"
benchmark_optimal(optimal_path)
likelihood_path = "likelihood.csv"
benchmark_likelihood(likelihood_path);

In [ ]:
clf_result_path = "../data/models/cross_entropy/ne_200_lr_0.001_bs_32/results.csv"

inf_result_paths = ["../data/models/b_0/ne_200_lr_1e-06_bs_1000_t_0.1/results.csv",
                    "../data/models/b_1/ne_200_lr_1e-06_bs_1000_t_0.1/results.csv",
                    "../data/models/b_2/ne_200_lr_1e-06_bs_1000_t_0.1/results.csv",
                    "../data/models/b_2_aux/ne_200_lr_1e-06_bs_1000_t_0.1/results.csv",
                    "../data/models/b_3_aux/ne_200_lr_1e-06_bs_1500_t_0.1/results.csv"]


In [ ]:
df = pd.read_csv(clf_result_path)
for result_path in inf_result_paths:
  df = df.append(pd.read_csv(result_path),ignore_index=True)

df = df.append(pd.read_csv(optimal_path), ignore_index=True)
df = df.append(pd.read_csv(likelihood_path), ignore_index=True)


 

In [ ]:
cols = ["common_path","b_0","b_1","b_2","b_2_aux", "b_3_aux"]
df_sub = df.loc[:, cols]

In [ ]:
quant_df = df_sub.groupby("common_path").quantile([0.36,0.5,0.84]).unstack().round(4)

In [ ]:
row_map = {"cross_entropy/" : "NN Classifier",
           "models/b_0/" : "\textsc{INFERNO} 0",
           "models/b_1/" : "\textsc{INFERNO} 1",
           "models/b_2/" : "\textsc{INFERNO} 2",
           "models/b_2_aux/" : "\textsc{INFERNO} 3",
           "models/b_3_aux/" : "\textsc{INFERNO} 4",
           "optimal" : "Optimal Classifier",
           "likelihood" : "Analytical Likelihood"
           }

quant_map = {0.36 : r"36%",
             0.5 : r"50%",
             0.84: r"84%"}
def rename_rows(row_name):
  for k,v in row_map.items():
    if k in row_name:
      return v
  else:
    return row_name
  
           
bench_map = {"b_0" : "Benchmark 0",
             "b_1" : "Benchmark 1",
             "b_2" : "Benchmark 2",
             "b_2_aux" : "Benchmark 3",
             "b_3_aux" : "Benchmark 4"}

In [ ]:
format_df = (quant_df.rename(bench_map,level=0,axis=1)
                     .rename(quant_map,level=1,axis=1)
                     .rename(rename_rows).loc[list(row_map.values())])
del format_df.index.name
format_df

In [ ]:
def formatter(row):
  median = row.loc[(slice(None),"50%")]
  upper_unc = row.loc[(slice(None),"84%")] - median
  lower_unc = median - row.loc[(slice(None),"36%")]
  median_str = median.round(2).apply("{0:.2f}".format)
  result = median_str.apply(lambda x: "$"+x)\
                     .str.cat(upper_unc.round(2).apply("{0:.2f}".format),sep="^{+")\
                     .apply(lambda x: x+"}")\
                     .str.cat(lower_unc.round(2).apply("{0:.2f}".format),sep="_{-")\
                     .apply(lambda x: x+"}$")
  if ("Optimal" in row.name) or ("Likelihood" in row.name):
    return median_str
  else:
    return result

In [ ]:
compact_df = format_df.apply(formatter, axis=1)

In [ ]:
compact_df

In [ ]:
with open("../paper/table.tex", "w") as f:
  f.write(compact_df.to_latex(escape=False))